# Basic Usage

`pytket-dqc` is a python package for the distribution of quantum circuits between interconnected quantum computers. Here we gather some basic usage of `pytket-dqc`, covering the main data structures and methods you will encounter. These main data structures include: `NISQNetwork`, the network on which a circuit will be distributed; `Circuit`, describing the circuit to be distributed; `Distribution`, which describes the distribution of a circuit onto a network; and `Distributor`, the parent class for a collection of techniques for generating a distribution.

## Networks

Networks of quantum servers are specified by two properties. The fist is the server coupling, detailing which servers are connected to which others. In `pytket-dqc` this is specified by a list of pairs of integers, where each pair signifies that there is a connection between those two servers. The second is the qubits each server contains. This is specified by a dictionary from the server to a list of qubits it contains. 

An example is given below, where blue lines indicate connections between servers, and red lines indicate connections between qubits within servers. The vertex labels are the indexes of the qubits.

In [ ]:
from pytket_dqc.networks import NISQNetwork

network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0,1,2,3], 1:[4,5,6], 2:[7,8,9,10,11], 3:[12,13]})
f = network.draw_nisq_network()

## Circuits

Note that the accepted gate set for `pytket-dqc` is CU1, Rz, and H. While restrictive, this is a universal gate set, and `pytket-dqc` includes some handy utilities for rebasing your circuit if it is not in the correct gate set initially.

In [ ]:
from pytket_dqc.utils import DQCPass
from pytket_dqc.circuits import HypergraphCircuit
from pytket.circuit.display import render_circuit_jupyter
from pytket import Circuit

circ = Circuit(4).CY(0,1).CZ(1,2).H(1).CX(1,0).CZ(1,3)
render_circuit_jupyter(circ)

DQCPass().apply(circ)
render_circuit_jupyter(circ)

## Distributors and Distributions

`pytket-dqc` includes several instances of the `Distributor` class, each of which convert a `Circuit` and a `NISQNetwork` into a `Distribution`. One example of an available `Distributor` is `PartitioningHeterogeneous`, which uses hypergraph partitioning and some refinement to distribute a circuit to a network, as in the following example.

A `Distribution` includes several useful methods, including the ability to calculate the cost of the distribution and to generate the corresponding `pytket` `Circuit`, each of which is demonstrated below.

In [ ]:
from pytket_dqc.distributors import PartitioningHeterogeneous

network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1,2], 2:[3,4]})
network.draw_nisq_network()

distribution = PartitioningHeterogeneous().distribute(circ, network)
print(f"Cost of distribution: {distribution.cost()}")
distributed_circ = distribution.to_pytket_circuit()
render_circuit_jupyter(distributed_circ)

## Circuit Generation

Once a placement has been obtained, it it possible to use pytket-dqc to generate a distributed circuit which implements the original circuit between several servers. There are two circuit generation methods which may be used according to the detail required. 

Let's consider the following simple circuit as an example.

In [ ]:
from pytket import OpType

circ = Circuit(2)
circ.add_gate(OpType.CU1, 1.0, [0, 1]).H(0).Rz(0.3,0).H(0).add_gate(OpType.CU1, 1.0, [0, 1])
render_circuit_jupyter(circ)

Which we will place on the following simple network

Once a distribution is determined, we can call `to_pytket_circuit` to generate the distributed circuit. The qubits are labelled according to their allocation to servers.

In [ ]:
from pytket_dqc.placement import Placement
from pytket_dqc import Distribution

hyp_circ = HypergraphCircuit(circ)
placement = Placement({0:1, 1:2, 2:0, 3:0})
distribution = Distribution(hyp_circ, placement, network)
assert distribution.is_valid()

circ_with_dist = distribution.to_pytket_circuit()
render_circuit_jupyter(circ_with_dist)

Notice that ancilla qubits are added to hold the e-bit. Morever, we see that the gates are acted on a different server from the servers where the qubits are placed. As such in this case we have one server which consists of only link qubits.

`pytket-dqc` will make some effort to use the minimum number of distribution operations required to implement a sequence of gates remotely, given a `Placement`. Consider for example the following three pronged star network.

In [ ]:
network = NISQNetwork([[0,1], [0,2], [0,3]], {0:[0], 1:[1], 2:[2], 3:[3]})
f = network.draw_nisq_network()

and the following simple three qubit circuit.

In [ ]:
circ = Circuit(3)
circ.add_gate(OpType.CU1, 1.0, [0, 1]).add_gate(OpType.CU1, 1.0, [0, 2])
render_circuit_jupyter(circ)

Imagine the qubits of this circuit are placed on the prongs of this network, and the gates on the servers that hold their target. Then the minimal way to distribute these gates is to copy the information contained in the control qubit to the central server, and then copy the information again from there to the two other edge servers. `pytket-dqc` will take this approach, as seen here.

In [ ]:
placement = Placement({0:1, 1:2, 2:3, 3:2, 4:3})
distribution = Distribution(HypergraphCircuit(circ), placement, network)
assert distribution.is_valid()

circ_with_dist = distribution.to_pytket_circuit()
render_circuit_jupyter(circ_with_dist)

More formally, `pytket-dqc` will consume e-bits along the edges of the [Steiner tree](https://en.wikipedia.org/wiki/Steiner_tree_problem) which connects the servers in which the qubits acted on by the sequence of gates reside.